# Install Libraries if Needed (Only Run Once)

In [ ]:
# !pip install -U deepxde

# Import Libraries

In [ ]:
# Interactive Plotting

# for jupyter notebooks
%matplotlib notebook 

# for jupyter labs
# %matplotlib widget 

In [ ]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch"""
import io
import re
import os

import matplotlib.pyplot as plt
import numpy as np

import deepxde as dde
from deepxde.backend import tf

from mpl_toolkits.mplot3d import Axes3D
import sys
from scipy.integrate import odeint
import matplotlib.tri as tri
from mpl_toolkits import mplot3d

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Laplace Equation with holes for anode and cathod
$$
u_{xx} + u_{yy} = 0, \quad x \in(-1,1), y \in(-1,1)\\
u(-1,y)=u(1,y)=u(x,-1)=u(x,1)=0\\
u((x+0.5)^2+y^2<r^2) = 1\\
u((x-0.5)^2+y^2<r^2) = -1\\
r = 0.1
$$


Example originally from https://github.com/lululxvi/deepxde/issues/141

# Define PDE

In [ ]:
def pde(x, y):
    dy_x = tf.gradients(y, x)[0]
    dy_x, dy_y = dy_x[:, 0:1], dy_x[:, 1:]
    dy_xx = tf.gradients(dy_x, x)[0][:, 0:1]
    dy_yy = tf.gradients(dy_y, x)[0][:, 1:]
    return -dy_xx - dy_yy

# Functions to Identify the Various Boundaries

In [ ]:
def boundary(_, on_boundary):
    return on_boundary

def boundary_outer(x, on_boundary):
    norm = np.sqrt(x[0]**2 + x[1]**2)
    return on_boundary and (norm > 0.9)

def boundary_cathode(x, on_boundary):
    return on_boundary & (x[0] < 0) & (x[0] > -0.9) & (x[1] < 0.9) & (x[1] > -0.9)

def boundary_anode(x, on_boundary):
    return on_boundary & (x[0] > 0) & (x[0] < 0.9) & (x[1] < 0.9) & (x[1] > -0.9)

# Value at boundaries
def value_outer(x):
    num_data = x.shape[0]
    return np.zeros((num_data,1))

def value_cathode(x):
    num_data = x.shape[0]
    return np.ones((num_data,1)) * 1

def value_anode(x):
    num_data = x.shape[0]
    return -np.ones((num_data,1)) * 1

# Geometry Operators 
Can combine geometries in various ways. Take to geometries geom1 and geom2
## Union (combine domain of geom1 and geom2 for points in either)
```python
geom = geom1 | geom2
```
## Intersection (domain of points in geom1 and geom2)
```python
geom = geom1 & geom2
```
## Difference (exclude geom2 from geom1)
```python
geom = geom1 - geom2
```
## More details
For more details, refer to the geometry class in deepxde https://github.com/lululxvi/deepxde/blob/master/deepxde/geometry/geometry.py



# Define Geometry and BCs

In [ ]:
radius = 0.1


bound = dde.geometry.Rectangle([-1,-1], [1,1])
cathode = dde.geometry.Disk([-0.5,0], radius)
anode = dde.geometry.Disk([0.5,0], radius)
geom = bound - cathode - anode
# geom = dde.geometry.CSGDifference(bound, cathode)
# geom = dde.geometry.CSGDifference(geom, anode)
bc_outer = dde.DirichletBC(geom, value_outer, boundary_outer)
bc_cathode = dde.DirichletBC(geom, value_cathode, boundary_cathode)
bc_anode = dde.DirichletBC(geom, value_anode, boundary_anode)

# Define PDE, Net, and Model

In [ ]:
data = dde.data.PDE(geom, pde, [bc_outer, bc_cathode, bc_anode], num_domain=60000, num_boundary=6000, num_test=15000)
net = dde.maps.FNN([2] + [50] * 4 + [1], "tanh", "Glorot uniform")
model = dde.Model(data, net)

# Define Paths

In [ ]:
save_dir = 'Complex_Geometry'
model_name = 'model'
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, model_name)
ckpt_path = tf.train.latest_checkpoint(save_dir)
loss_fname = 'loss_complex_geometry.dat'
train_fname = 'train_complex_geometry.dat'
test_fname = 'test_complex_geometry.dat'
loss_path = os.path.join(save_dir, loss_fname)
train_path = os.path.join(save_dir, train_fname)
test_path = os.path.join(save_dir, test_fname)

# ckpt_path = os.path.join(save_dir, 'checkpoint')
# model.save(model_path)

# Load Model

In [ ]:
model.compile("adam", lr=1e-3)
model.restore(ckpt_path, verbose=1)

# Train the Network

In [ ]:
# Note that we can adjust the loss weights.  Here the PDE loss is has a weight of 1, while the BCs each have a weight of 50
model.compile("adam", lr=0.001, loss_weights = [1, 50, 50, 50])
losshistory, train_state = model.train(epochs=25000)
model.compile("L-BFGS-B")
losshistory, train_state = model.train()

# Save Model

In [ ]:
model.save(model_path)

# Plot Training Performance

In [ ]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True, loss_fname=loss_path, train_fname=train_path, test_fname=test_path)

# Get Predictions on Test Data

In [ ]:

u_pred = model.predict(data.test_x).flatten()

In [ ]:
x = data.test_x[:,0]
y = data.test_x[:,1]


# Plot Results

In [ ]:

fig = plt.figure(figsize=(7,5))

triang = tri.Triangulation(x, y)
triang.set_mask((np.hypot((x[triang.triangles].mean(axis=1) - 0.5), y[triang.triangles].mean(axis=1)) < radius ) | (np.hypot((x[triang.triangles].mean(axis=1) + 0.5), y[triang.triangles].mean(axis=1)) < radius))

# plt.pcolor(data.test_x, u_pred, cmap='jet', shading='gouraud')
tpc = plt.tripcolor(triang, u_pred, cmap='jet', shading='gouraud')

plt.colorbar(tpc)
plt.axis('square')

plt.xlim([-1,1])
plt.ylim([-1,1])
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('$u_{pred}$')


plt.show()

In [ ]:
fig = plt.figure(figsize =(16, 9)) 
ax = plt.axes(projection ='3d') 
trisurf = ax.plot_trisurf(x, y, u_pred,
                         cmap = 'jet',
                         linewidth = 0.2,
                         antialiased = True,
                         edgecolor = 'grey') 
fig.colorbar(trisurf, ax = ax, shrink = 0.5, aspect = 5)
ax.set_title('Predictions', fontweight='bold')
# Adding labels
ax.set_xlabel('x', fontweight ='bold')
ax.set_ylabel('y', fontweight ='bold')
ax.set_zlabel('u', fontweight ='bold')
plt.show()